# Uncertainty in Autonomous Systems
Testing uncertainty in autonomous systems using the Carla simulator.

## Connect to Carla server

In [2]:
import carla
import random

In [10]:
# Connect to the Carla server. IP address below are used because of WSL2 networking.
client = carla.Client('172.22.112.1', 2000)
world = client.get_world()

## Manipulate environment

In [14]:
client.load_world('Town01')

In [22]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

# Set the spectator with an empty transform
spectator.set_transform(carla.Transform())
# This will set the spectator at the origin of the map, with 0 degrees
# pitch, yaw and roll - a good way to orient yourself in the map

## Add actors

In [38]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

# Show a few examples from the list
print(*list(vehicle_blueprints)[0:5], sep='\n')

ActorBlueprint(id=vehicle.audi.a2,tags=[a2, audi, vehicle])
ActorBlueprint(id=vehicle.citroen.c3,tags=[c3, citroen, vehicle])
ActorBlueprint(id=vehicle.chevrolet.impala,tags=[impala, chevrolet, vehicle])
ActorBlueprint(id=vehicle.dodge.charger_police_2020,tags=[charger_police_2020, dodge, vehicle])
ActorBlueprint(id=vehicle.micro.microlino,tags=[microlino, micro, vehicle])


In [27]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [39]:
# Add the ego vehicle, which will be the centerpoint of the simulation
ego_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [43]:
print(ego_vehicle.get_transform())
spectator.set_transform(ego_vehicle.get_transform())

Transform(Location(x=338.979980, y=226.750000, z=0.003931), Rotation(pitch=0.000000, yaw=-90.000267, roll=0.000000))


## Add sensors

In [40]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)